In [41]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import os
import glob
import cv2 # 专用于处理计算机视觉场景文件的包 pip install opencv-python 执行安装
import json
import time
# 查询系统可用的 GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
# 确保有可用的 GPU 如果没有, 则会报错
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
# 设置参数,该段务必在运行jupyter的第一段代码执行，否则会无法初始化成功
# 仅在需要时申请显存空间（程序初始运行时消耗很少的显存，随着程序的运行而动态申请显存）
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [42]:
fileList = glob.glob('/data/python/tensorflow/shuffle_data_gzip/*.csv.gz')

In [43]:
# 单通道作图函数定义
def draw_cv2(raw_strokes, BASE_SIZE = 256, size = 256, lw = 6):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)  # 定义画板
    for stroke in raw_strokes:
        for i in range(len(stroke[0]) - 1):
            _ = cv2.line(img, (stroke[0][i],stroke[1][i]),
                (stroke[0][i+1],stroke[1][i+1]), 255, lw
            )
    
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

In [44]:
if os.path.isfile('/data/python/tensorflow/tfrecord_data/train.tfrecords') == False:
    os.mkdir('/data/python/tensorflow/tfrecord_data/')
    open('/data/python/tensorflow/tfrecord_data/train.tfrecords',mode = 'a').close()

os.listdir('/data/python/tensorflow/tfrecord_data/')

['train.tfrecords']

In [45]:
tfrecord_file = '/data/python/tensorflow/tfrecord_data/train.tfrecords'

In [46]:
with tf.io.TFRecordWriter(tfrecord_file) as writer:
    for filename in fileList[:1]:
        df = pd.read_csv(filename)
        df['drawing'] = df['drawing'].apply(json.loads)
        for row in range(df.shape[0]):
            drawing = df.loc[row, 'drawing']
            img = draw_cv2(drawing, BASE_SIZE = 128, size = 128, lw =6)
            img = img.tostring()
            label = df.loc[row, 'y']

            feature = {
                'image':tf.train.Feature(bytes_list = tf.train.BytesList(value = [img])), # 图片是一个bytes对象
                'label':tf.train.Feature(int64_list = tf.train.Int64List(value = [label])) # 标签是一个Int对象
            }

            example = tf.train.Example(features = tf.train.Features(feature = feature)) # 通过字典建立Example
            writer.write(example.SerializeToString()) # 将Example序列化并写入TFRecord文件

In [59]:
# 读取TFRecord文件

raw_dataset = tf.data.TFRecordDataset(tfrecord_file)

# 定义Feature结构，告诉解码器每个Feature的类型是什么

feature_description = {
    'image':tf.io.FixedLenFeature([], tf.string),
    'label':tf.io.FixedLenFeature([], tf.int64)
}

# 将TFRecord文件中的每一个序列化的tf.train.Example解码


def _parse_example(example_string): # 将TFRecord 文件中的每一个序列化的 tf.train.Example 解码
    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    image = tf.io.decode_raw(feature_dict['image'], tf.uint8) # 解码JPEG图片
    image = tf.reshape(image, [128,128,1])
    image = tf.dtypes.cast(image, tf.float32)
    image = image /255.0
    label = tf.one_hot(feature['label'],depth = 340)

    return image, label

In [60]:
dataset = raw_dataset.map(_parse_example)

TypeError: in user code:

    <ipython-input-53-f528563dfe83>:21 _parse_example  *
        label = tf.one_hot(feature['label'],depth = 340)
    /home/hp/.local/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:180 wrapper  **
        return target(*args, **kwargs)
    /home/hp/.local/lib/python3.8/site-packages/tensorflow/python/ops/array_ops.py:4009 one_hot
        return gen_array_ops.one_hot(indices, depth, on_value, off_value, axis,
    /home/hp/.local/lib/python3.8/site-packages/tensorflow/python/ops/gen_array_ops.py:6198 one_hot
        _, _, _op, _outputs = _op_def_library._apply_op_helper(
    /home/hp/.local/lib/python3.8/site-packages/tensorflow/python/framework/op_def_library.py:473 _apply_op_helper
        raise err
    /home/hp/.local/lib/python3.8/site-packages/tensorflow/python/framework/op_def_library.py:465 _apply_op_helper
        values = ops.convert_to_tensor(
    /home/hp/.local/lib/python3.8/site-packages/tensorflow/python/framework/ops.py:1341 conver

In [ ]:
train_ds = dataset.prefetch(buffer_size = tf.data.experimental.AUTOTUNE).shuffle(buffer_size = tf.data.experimental.AUTOTUNE).batch(1024)

#### 同5_3_2模型调用部分相同

In [ ]:
class MobileNetModel(tf.keras.models.Model):
    def __init__(self, size, n_labels, **kwargs):
        super(MobileNetModel, self).__init__(**kwargs)
        self.base_model = tf.keras.applications.MobileNet(
                input_shape = (size, size, 1),
                include_top = False,
                weights=None,
                classes = n_labels
            )
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(1024, activation='relu')
        self.outputs = tf.keras.layers.Dense(n_labels, activation='softmax')


    def call(self, inputs):
        x = self.base_model(inputs)
        x = self.flatten(x)
        x = self.dense(x)
        output_ = self.outputs(x)
        return output_

In [ ]:
model = MobileNetModel(size = 64, n_labels = NCATS)

loss_object = tf.keras.losses.CategoricalCrossentropy()

learning_rate = 0.002
optimizer = tf.keras.optimizers.Adam(learning_rate= learning_rate)

train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy('train_accuracy')
train_top3_accuracy = tf.keras.metrics.TopKCategoricalAccuracy(k =3 , name = 'train_top_3_categorical_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy('test_accuracy')
test_top3_accuracy = tf.keras.metrics.TopKCategoricalAccuracy(k =3 , name = 'test_top_3_categorical_accuracy')

def train_one_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    train_top3_accuracy(labels, predictions)

def val_one_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    test_top3_accuracy(labels, predictions)

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    # 在下一个epoch开始的时候，重置评估指标
    train_loss.reset_states()
    train_accuracy.reset_states()
    train_top3_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    test_top3_accuracy.reset_states()

    for step,(images, labels) in enumerate(train_ds):
        train_one_step(images, labels)

        if step %200 == 0:
             print('step :{0}, Samples: {1}, Train Loss: {2}, Train Accuracy: {3}, Train Top3 Accuracy: {4}'.format(
                 step, (step + 1) * 1024, train_loss.result(), train_accuracy.result() * 100,
                 train_top3_accuracy.result() * 100
             ))

        if step > 1000:
            break

        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(
        epoch + 1,
        train_loss.result(),
        train_accuracy.result() * 100,
        train_top3_accuracy.result() * 100,
        test_loss.result(),
        test_accuracy.result() * 100,
        test_top3_accuracy.result() * 100
    ))